In [2]:
import torch 
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

transform_train = transforms.Compose([transforms.Resize((32,32)),  #resises the image so it can be perfect for our model.
                                      transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
                                      transforms.RandomRotation(10),     #Rotates the image to a specified angel
                                      transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
                                      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Set the color params
                                      transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #Normalize all the images
                               ])

transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ["plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]



################################################################################################### define the CNN architecture
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv_layer = nn.Sequential(
        nn.Conv2d(3, 6, 3, padding = 1),
        nn.BatchNorm2d(6),
        nn.ReLU(),
        nn.Conv2d(6, 10, 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2),#

        nn.Conv2d(10, 20, 3, padding = 1),
        nn.BatchNorm2d(20),
        nn.ReLU(),
        nn.Conv2d(20, 40, 3, padding = 2),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout(0.1),#

        nn.Conv2d(40, 80, 3, padding = 2),
        nn.BatchNorm2d(80),
        nn.ReLU(),
        nn.Conv2d(80, 160, 3, padding = 2),
        nn.ReLU(),
        nn.MaxPool2d(2))
    
    self.fc_layer = nn.Sequential(
        nn.Dropout(0.1),
        nn.Flatten(),
        nn.Linear(160*6*6, 1024),
        nn.ReLU(),
        nn.Linear(1024, 128),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(128, 10))

    
  def forward(self, x):
    x = self.conv_layer(x)
    x = self.fc_layer(x)
    return x

# create a complete CNN
model = Net()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()



####################################################################################################################################
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)



n_epochs = [*range(2)]

for epoch in range(1, len(n_epochs)+1):

    train_loss = 0.0
    model.train()

    for data, target in trainloader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} '.format(
        epoch, train_loss))
   


CUDA is available!  Training on GPU ...
Files already downloaded and verified
Files already downloaded and verified
Net(
  (conv_layer): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(6, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Dropout(p=0.1, inplace=False)
    (13): Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (14): BatchNorm2d(80, e

In [3]:

class_correct = {x: 0.0 for x in classes}
total_correct = 0
class_total = {x: 0.0 for x in classes}

model.eval()

for data, target in testloader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()

    output = model(data)    
    pred = output.argmax(1)    

    for out, prediction in zip(target, pred):
        class_total[classes[out]] += 1
        if (out == prediction):
            total_correct += 1
            class_correct[classes[out]] += 1 
    
test_size = len(testloader.dataset)

print(f"Total accuracy of tests: {(total_correct / test_size)*100}% ")
print("Accuracy of each class: \n")
for classs in classes:
    print(f"Accuracy of {classs}: {(class_correct[classs] / class_total[classs])*100}%")



Total accuracy of tests: 44.66% 
Accuracy of each class: 

Accuracy of plane: 52.900000000000006%
Accuracy of car: 72.0%
Accuracy of bird: 44.6%
Accuracy of cat: 19.3%
Accuracy of deer: 24.6%
Accuracy of dog: 31.1%
Accuracy of frog: 51.1%
Accuracy of horse: 56.2%
Accuracy of ship: 63.800000000000004%
Accuracy of truck: 31.0%
